# Llama iheritune

In [1]:
def count_parameters(model):
    total_params = sum(p.numel() for p in model.parameters()) / 10**6
    print(f'total_params: {total_params:.3f}M')

In [2]:
!git clone https://github.com/EleutherAI/lm-evaluation-harness

Cloning into 'lm-evaluation-harness'...
remote: Enumerating objects: 34143, done.
remote: Counting objects: 100% (252/252), done.
remote: Compressing objects: 100% (140/140), done.
remote: Total 34143 (delta 150), reused 189 (delta 109), pack-reused 33891
Receiving objects: 100% (34143/34143), 23.17 MiB | 19.15 MiB/s, done.
Resolving deltas: 100% (23884/23884), done.


In [3]:
cd lm-evaluation-harness

/kaggle/working/lm-evaluation-harness


In [4]:
!git checkout 115206dc89dad67b8beaa90051fb52db77f0a529

Note: switching to '115206dc89dad67b8beaa90051fb52db77f0a529'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

HEAD is now at 115206dc Merge pull request #1005 from Andrei-Aksionov/fix_indent_lm_eval_tasks_bigbench


In [5]:
!pip install -e .

Obtaining file:///kaggle/working/lm-evaluation-harness
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 1.9 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 5.3 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 37.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.1/111.1 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.4/199.4 kB 10.1 MB/s eta 0:00:00
  Created wheel for antlr4-python3-run

In [7]:
pwd

'/kaggle/working/lm-evaluation-harness'

In [ ]:
# !pip install lm_eval

In [ ]:
# !pip install evaluate

In [8]:
import json
import sys
from pathlib import Path
from typing import Dict, List, Literal, Optional

from lm_eval import evaluator, tasks

import numpy as np
import torch
from torch.nn import Linear, CrossEntropyLoss
from datasets import load_dataset, concatenate_datasets
from transformers import BertTokenizer, BertModel, default_data_collator, pipeline, AutoModelForSequenceClassification
from transformers import LlamaForCausalLM, LlamaTokenizer, AutoConfig
from torch.utils.data import DataLoader
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

from lm_eval.base import BaseLM

2024-05-30 09:10:21.344676: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-30 09:10:21.344774: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-30 09:10:21.614573: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [9]:
parent_llama = LlamaForCausalLM.from_pretrained("openlm-research/open_llama_3b_v2")

count_parameters(parent_llama)

config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/6.85G [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

total_params: 3426.474M


In [10]:
def preprocess_function2(examples, tokenizer):
    return tokenizer(examples['text'], return_special_tokens_mask=True, padding='max_length', truncation=True, max_length=128)

In [9]:
wikipedia_dataset = load_dataset('wikipedia', '20220301.en', split='train[:1%]')
bookcorpus_dataset = load_dataset('bookcorpus', split='train[:1%]')
combined_dataset = concatenate_datasets([wikipedia_dataset, bookcorpus_dataset])

Generating train split:   0%|          | 0/74004228 [00:00<?, ? examples/s]

In [ ]:
map_tokenizer=LlamaTokenizer.from_pretrained("openlm-research/open_llama_3b_v2")
map_tokenizer.add_special_tokens({'pad_token': '[PAD]'})
combined_encoded_dataset = combined_dataset.map(lambda examples: preprocess_function2(examples, map_tokenizer), batched=True)

Map:   0%|          | 0/804629 [00:00<?, ? examples/s]

In [ ]:
print("upd1")

Наследуем student model

In [11]:
parent_config = AutoConfig.from_pretrained("openlm-research/open_llama_3b_v2")

In [12]:
print(parent_config)

LlamaConfig {
  "_name_or_path": "openlm-research/open_llama_3b_v2",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 3200,
  "initializer_range": 0.02,
  "intermediate_size": 8640,
  "max_position_embeddings": 2048,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 26,
  "num_key_value_heads": 32,
  "pad_token_id": 0,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "tie_word_embeddings": false,
  "torch_dtype": "float16",
  "transformers_version": "4.39.3",
  "use_cache": true,
  "vocab_size": 32000
}



In [14]:
child_config = AutoConfig.from_pretrained("openlm-research/open_llama_3b_v2", num_hidden_layers=13)

In [15]:
print(child_config)

LlamaConfig {
  "_name_or_path": "openlm-research/open_llama_3b_v2",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 3200,
  "initializer_range": 0.02,
  "intermediate_size": 8640,
  "max_position_embeddings": 2048,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 13,
  "num_key_value_heads": 32,
  "pad_token_id": 0,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "tie_word_embeddings": false,
  "torch_dtype": "float16",
  "transformers_version": "4.39.3",
  "use_cache": true,
  "vocab_size": 32000
}



In [26]:
import gc
gc.collect()
del child_llama

In [27]:
child_llama = LlamaForCausalLM(child_config)
# child_llama = LlamaForCausalLM.from_pretrained("openlm-research/open_llama_3b_v2", config=child_config)

for i in range(13):
    child_llama.model.layers[i].load_state_dict(parent_llama.model.layers[i].state_dict())

In [29]:
class ModelWrapper(BaseLM):
    def __init__(
        self,
        model,
        batch_size,
        tokenizer,
        device
    ):
        super().__init__()
        self.model = model
        self.model.to(device)
        self.tokenizer = tokenizer
        self.batch_size_per_gpu = batch_size
        self.device_ = device

    @torch.inference_mode()
    def _model_call(self, inps):
        outputs = self.model(inps)
        if hasattr(outputs, 'logits'):
            return outputs.logits
        elif hasattr(outputs, 'last_hidden_state'):
            return outputs.last_hidden_state
        else:
            raise ValueError("Model output does not contain 'logits' or 'last_hidden_state'")

    @torch.inference_mode()
    def _model_generate(self, context, max_length, eos_token_id) -> torch.Tensor:
        # this only supports batch size 1
        assert context.shape[0] == 1
        out = generate(self.model, context[0], max_length, eos_id=eos_token_id)
        for block in self.model.transformer.h:
            block.attn.kv_cache.reset_parameters()
        return out.unsqueeze(0)

    @property
    def batch_size(self):
        return self.batch_size_per_gpu*2

    @property
    def device(self):
        return self.device_

    @property
    def eot_token_id(self):
        # we use EOT because end of *text* is more accurate for what we're doing than end of *sentence*
        return self.tokenizer.eos_id

    @property
    def max_gen_toks(self):
        return 256

    @property
    def max_length(self):
        return self.model.config.max_position_embeddings

    def tok_encode(self, string: str) -> List[int]:
        return self.tokenizer.encode(string)

    def tok_decode(self, tokens: List[int]) -> str:
        t = torch.tensor(tokens)
        return self.tokenizer.decode(t)

In [30]:
eval_tasks: List[str] = ['piqa']

BATCH_SIZE = 2
num_fewshot = 0
limit = 256
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
tokenizer=LlamaTokenizer.from_pretrained("openlm-research/open_llama_3b_v2")

wrapped_model = ModelWrapper(
    model=child_llama,
    batch_size=BATCH_SIZE,
    tokenizer=tokenizer,
    device=device
)

tokenizer_config.json:   0%|          | 0.00/593 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/512k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/330 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [32]:
results = evaluator.evaluate(
    lm=wrapped_model,
    task_dict=tasks.get_task_dict(eval_tasks),
    num_fewshot=num_fewshot,
    limit=limit
)

Task: piqa; number of docs: 1838
Task: piqa; document 0; context prompt (starting on next line):
Question: Remove seeds from  strawberries
Answer:
(end of prompt on previous line)
Requests: [Req_loglikelihood('Question: Remove seeds from  strawberries\nAnswer:', ' Blend the strawberries, pour the mixture through a fine-mesh strainer with a bowl underneath to catch the pulps and strain out the seeds')[0]
, Req_loglikelihood('Question: Remove seeds from  strawberries\nAnswer:', ' Chop up the strawberries, pour the mixture through a fine-mesh strainer with a bowl underneath to catch the pulps and strain out the seeds')[0]
]
Running loglikelihood requests


100%|██████████| 512/512 [00:25<00:00, 19.92it/s]


In [33]:
print(results)

{'results': {'piqa': {'acc': 0.48046875, 'acc_stderr': 0.03128731687362685, 'acc_norm': 0.4921875, 'acc_norm_stderr': 0.03130739215119687}}, 'versions': {'piqa': 0}}


Выводы

Метод Inheritune:

Преимущества: Быстрое обучение, меньше данных для обучения.

Недостатки: Ограниченная гибкость архитектуры, зависит от качества и объема данных.

Метод дистилляции знаний:

Преимущества: Высокая точность, особенно при наличии большого объема данных.

Недостатки: Высокие вычислительные затраты, сложность настройки.

Когда использовать:

Inheritune: Подходит для случаев с ограниченными вычислительными ресурсами и данными. Идеально для быстрого прототипирования и ситуаций, когда необходима производительность на устройствах с ограниченными ресурсами.
Дистилляция знаний: Подходит для случаев, когда требуется высокая точность и есть доступ к большому объему данных и вычислительных ресурсов. Идеально для конечных моделей в производственных условиях.